# Integrating XML with NLTK

[NLTK](http://www.nltk.org/), the Python Natural Languge ToolKit package, is designed to work with plain text input, but sometimes your input is in XML. There are two principal paths to reconciliation: either use an XML environment that supports NLP (natural language processing) or let Python (which supports NLP through NLTK) manage the XML. The first approach, sticking to an XML environment, is illustrated in Week 3 of the Institute in the context of the [eXist XML database](http://exist-db.org), which integrates the [Stanford Core NLP tools](https://stanfordnlp.github.io/CoreNLP/). Here we illustrate the second approach, letting Python manage the XML.

## Before you make a mistake

It’s natural to think of *parsing* (reading, interpreting, and processing) XML with regular expressions, but it’s also Wrong for at least two sets of reasons:

1. Regular expressions operate over strings, and there are string differences in XML that are not informationally different. For example, the order of attributes on an element, whether the attributes are single- or double-quoted, whether a Unicode character is represented by a raw character or a numerical character reference, and many other details represent string differences that are not informational differences. The same is true of the extent and type of white space *in some environments but not others*. And the same is true when you have to recognize whether a right angle bracket or a single or double quotation mark is part of content or part of markup. XML-aware processing knows what’s informational and what isn’t, as well as what’s content and what’s markup. You don’t want to reinvent *those* wheels.

1. Parsing XML is a recursive operation. For example, if you have two elements of the same type nested inside each other, as in

```xml
<emphasis><emphasis>a very emphatic thought</emphasis></emphasis>
```
parsing has to match up the correctly paired start and end tags. XML-aware processing knows where it is in the tree. That’s another wheel you don’t want to reinvent.

It’s also natural to think of *writing* XML by constructing a string, such as concatenating angle brackets and text and other bits and pieces. This is a Bad Idea because some decisions are context sensitive, and keeping track of the context is challenging. For example, attribute values can be quoted with single or double quotation marks, but if the value contains a single or double quotation mark, that can influence the choice, and there are situations where you may need to represent the quotation marks in attribute values with `&quot;` or `&apos;` character entities instead of as raw characters. A library that knows how to write XML will keep track of that for you.

## Wrangling XML in Python

The Python Standard Library provides several [tools for parsing and creating XML](https://docs.python.org/3/library/markup.html), and there are also third-party packages. In this tutorial we use two parts of the Standard Library: [`pulldom`](https://docs.python.org/3/library/xml.dom.pulldom.html) for parsing XML input and [`minidom`](https://docs.python.org/3/library/xml.dom.minidom.html) for constructing XML output. You can read more about these modules by clicking on the preceding links to the Standard Library documentation, and also in the [Structured text: XML](http://etutorials.org/Programming/Python+tutorial/Part+IV+Network+and+Web+Programming/Chapter+23.+Structured+Text+XML/) chapter of the eTutorials.org [Python tutorial](http://etutorials.org/Programming/Python+tutorial/).

Initially we’ll use use a small input XML document, tag each word as a `<word>` element, and add part of speech (POS) and lemma (dictionary form) attributes to the words. We’ll use `pulldom` to read, parse, and process the input document, NLTK to determine the part of speech and the lemma, and `minidom` to create the output.

## Input XML

Create the following small XML document in a work directory and save with a filename like `test.xml`:

```xml
<root>
    <p speaker="hamlet">Hamlet is a thane of Denmark.</p>
    <p speaker='ophelia'>Things end badly for Ophelia.</p>
    <p speaker="nobody">Julius Caesar does not appear in this play.</p>
</root>
```

## Desired output XML

The desired output is:

```xml
<?xml version="1.0" ?>
<root>
	<p speaker="hamlet">
		<word lemma="hamlet" pos="NNP">Hamlet</word>
		<word lemma="be" pos="VBZ">is</word>
		<word lemma="a" pos="DT">a</word>
		<word lemma="thane" pos="NN">thane</word>
		<word lemma="of" pos="IN">of</word>
		<word lemma="denmark" pos="NNP">Denmark</word>
		<word lemma="." pos=".">.</word>
	</p>
	<p speaker="ophelia">
		<word lemma="thing" pos="NNS">Things</word>
		<word lemma="end" pos="VBP">end</word>
		<word lemma="badly" pos="RB">badly</word>
		<word lemma="for" pos="IN">for</word>
		<word lemma="ophelia" pos="NNP">Ophelia</word>
		<word lemma="." pos=".">.</word>
	</p>
	<p speaker="nobody">
		<word lemma="julius" pos="NNP">Julius</word>
		<word lemma="caesar" pos="NNP">Caesar</word>
		<word lemma="do" pos="VBZ">does</word>
		<word lemma="not" pos="RB">not</word>
		<word lemma="appear" pos="VB">appear</word>
		<word lemma="in" pos="IN">in</word>
		<word lemma="this" pos="DT">this</word>
		<word lemma="play" pos="NN">play</word>
		<word lemma="." pos=".">.</word>
	</p>
</root>
```

## The python code

Here is the entire Python script that creates the output (we describe how the pieces work below). If you have saved the sample input as `text.xml` in the same directory as the location of this notebook, you can run the transformation in the notebook now, and the output should be displayed below:

In [1]:
#!/usr/bin/env python
"""Tag words and add POS and lemma information in XML document."""

from xml.dom.minidom import Document
from xml.dom import pulldom
import nltk


# https://github.com/rhdekker/python_xml_pull_parser_example/blob/master/pull_parser_test.py
class Stack(list):
    def push(self, item):
        self.append(item)

    def peek(self):
        return self[-1]


# https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
def get_wordnet_pos(treebank_tag: str) -> str:
    """Replace treebank POS tags with wordnet ones."""
    if treebank_tag.startswith('J'):
        return nltk.corpus.reader.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.reader.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.reader.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.reader.wordnet.ADV
    else:
        return 'n'


def lemmatize(text: str, pos: str) -> str:
    return nltk.stem.WordNetLemmatizer().lemmatize(text.lower(), get_wordnet_pos(pos))


def extract(input_xml):
    """Process entire input XML document, firing on events"""
    # Initialize output as XML tree, stack of open elements
    d = Document()
    open_elements = Stack()
    open_elements.push(d)
    # Start pulling; it continues automatically
    doc = pulldom.parse(input_xml)
    for event, node in doc:
        if event == pulldom.START_ELEMENT:
            open_elements.peek().appendChild(node)
            open_elements.push(node)
        elif event == pulldom.END_ELEMENT:
            open_elements.pop()
        elif event == pulldom.CHARACTERS:
            value = node.toxml()
            words = nltk.word_tokenize(value)
            pos_tagged = nltk.pos_tag(words)
            for (text, pos) in pos_tagged:
                word = d.createElement("word")
                word.setAttribute("pos", pos)
                word.setAttribute("lemma", lemmatize(text, pos))
                t = d.createTextNode(text)
                word.appendChild(t)
                open_elements.peek().appendChild(word)
        else:
            continue
    return d


with open('test.xml', 'rb') as test_in:
    results = extract(test_in)
    print(results.toprettyxml())

<?xml version="1.0" ?>
<root>
	<p speaker="hamlet">
		<word lemma="hamlet" pos="NNP">Hamlet</word>
		<word lemma="be" pos="VBZ">is</word>
		<word lemma="a" pos="DT">a</word>
		<word lemma="thane" pos="NN">thane</word>
		<word lemma="of" pos="IN">of</word>
		<word lemma="denmark" pos="NNP">Denmark</word>
		<word lemma="." pos=".">.</word>
	</p>
	<p speaker="ophelia">
		<word lemma="thing" pos="NNS">Things</word>
		<word lemma="end" pos="VBP">end</word>
		<word lemma="badly" pos="RB">badly</word>
		<word lemma="for" pos="IN">for</word>
		<word lemma="ophelia" pos="NNP">Ophelia</word>
		<word lemma="." pos=".">.</word>
	</p>
	<p speaker="nobody">
		<word lemma="julius" pos="NNP">Julius</word>
		<word lemma="caesar" pos="NNP">Caesar</word>
		<word lemma="do" pos="VBZ">does</word>
		<word lemma="not" pos="RB">not</word>
		<word lemma="appear" pos="VB">appear</word>
		<word lemma="in" pos="IN">in</word>
		<word lemma="this" pos="DT">this</word>
		<word lemma="play" pos="NN">play</word>
		<wo

## How it works

We’ve divided the program into sections below, with explanations after each section.

In [2]:
#!/usr/bin/env python
"""Tag words and add POS and lemma information in XML document."""

'Tag words and add POS and lemma information in XML document.'

A Python program begins with a *shebang* and a [*docstring*](https://www.python.org/dev/peps/pep-0257/). The shebang makes it easier to run the program from the command line, and the *docstring* documents what the program does, The shebang must be the very first line in a program. For now, think of the shebang as a magic incantation that should be copied and pasted verbatim; we explain below what it means. The docstring should be a single line framed by triple single quotation marks, and it should describe concisely what the program does. When you execute the docstring by itself, as we do above, it echoes itself to the screen; when you run the program, though, it remains silent.

In [3]:
from xml.dom.minidom import Document
from xml.dom import pulldom
import nltk

We import the ability to create a new XML document, which we’ll use to create our output, from `minidom`, and we import `pulldom` to parse the input document. We import `nltk` because we’ll use it to determine the part of speech and lemma information for each word.

In [4]:
# https://github.com/rhdekker/python_xml_pull_parser_example/blob/master/pull_parser_test.py
class Stack(list):
    def push(self, item):
        self.append(item)

    def peek(self):
        return self[-1]

When you have to look up how to do something, it’s smart to include the source in a comment, so that you can find it again if you need to check anything. It’s also a way of giving credit to other developers. Python comments begin with a hash mark.

We define a type of object called a `Stack` which inherits all of the properties of a Python `list`, plus two new *methods*. You can read about [classes in Python in the Standard Library documentation](https://docs.python.org/3/tutorial/classes.html). We explain below how this class and its methods work when we use them. It doesn’t do anything when we define it; it just hangs around until we use it.

In [5]:
# https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
def get_wordnet_pos(treebank_tag: str) -> str:
    """Replace treebank POS tags with wordnet ones."""
    if treebank_tag.startswith('J'):
        return nltk.corpus.reader.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nltk.corpus.reader.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nltk.corpus.reader.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nltk.corpus.reader.wordnet.ADV
    else:
        return nltk.corpus.reader.wordnet.NOUN

We create a function called `get_wordnet_pos()`, which we’ll use later. This function is defined as taking a treebank tag (a convention for identifying parts of speech used by the NLTK part of speech tagger), which is a string, and it returns a value that is also a string. The reason we need to do this is that the NLTK part of speech tagger uses one set of part of speech identifiers, but Wordnet, the NLTK component that performs lemmatization, uses a different one. Since we do the part of speech tagging first, we use this function to convert that value to one that Wordnet will understand before we perform lemmatization. There are many treebank part of speech tags but only four Wordnet ones, for nouns, verbs, adjectives, and adverbs, and everything else is treated as a noun. Since our lemmatization routine always supplies a part of speech identifier to Wordnet, our function returns the correct value for the four defined parts of speech and efaults to the value for nouns otherwise.

We could have made this a bit more concise by not defining the treatment of nouns, and just letting them fall into the `else` condition, but the extra two lines make our intentions a little clearer. Like the class definition above, this function doesn’t do anything when we define it, but it’s available for us to use later.

In [6]:
def lemmatize(text: str, pos: str) -> str:
    return nltk.stem.WordNetLemmatizer().lemmatize(text.lower(), get_wordnet_pos(pos))

We define a function called `lemmatize()` that takes two pieces of input, both of which are strings, and returns a string. The parameter `text` is the word to be lemmatized and the parameter `pos` is the part of speech in treebank form. We call the NLTK function to identify the lemma with `nltk.stem.WordNetLemmatizer().lemmatize()` with two arguments. The lemmatizer expects words to be lower case, so we convert the `text` to lower case with the `lower()` string method. And it requires a Wordnet part of speech, and not a treebank one, so we use our `get_wordnet_pos()` function to perform the conversion.

In [7]:
def extract(input_xml):
    """Process entire input XML document, firing on events"""
    # Initialize output as XML tree, stack of open elements
    d = Document()
    open_elements = Stack()
    open_elements.push(d)
    # Start pulling; it continues automatically
    doc = pulldom.parse(input_xml)
    for event, node in doc:
        if event == pulldom.START_ELEMENT:
            open_elements.peek().appendChild(node)
            open_elements.push(node)
        elif event == pulldom.END_ELEMENT:
            open_elements.pop()
        elif event == pulldom.CHARACTERS:
            value = node.toxml()
            words = nltk.word_tokenize(value)
            pos_tagged = nltk.pos_tag(words)
            for (text, pos) in pos_tagged:
                word = d.createElement("word")
                word.setAttribute("pos", pos)
                word.setAttribute("lemma", lemmatize(text, pos))
                t = d.createTextNode(text)
                word.appendChild(t)
                open_elements.peek().appendChild(word)
        else:
            continue
    return d

Our `extract()` function does all the work, calling on the functions we defined earlier as needed. Here’s how it works (with line numbers):

* **1:** `extract()` is a function that gets called with one argument, which we assign to a parameter we’ve called `input_xml`.
* **4:** Near the top of the full program we used `from xml.dom.minidom import Document` to make the Document class available to our program. Here we use it to create a new XML document, which we assign as the value of a new variable `d`. We’ll use this to build our output document.
* **5:** Our `open_elements` variable is an *instance* of our `Stack` class. We use it to keep track of where we are in the tree.
* **6:** The first value in `open_elements` is our document. A stack is a FIFO (first in, first out) structure, which means that we add items to the end of the list with `push()` and remove them from the end of the list with `pop()`. When we defined our Stack class, we also made it possible to access the most recent item pushed onto the list with our `peek()` method.
* **8:** `pulldom` is a streaming parser, which means that once we start processing elements in the XML input tree, the parser performs a traversal that visits every node of the tree. We start that process with `pulldom.parse()`, telling it to parse the document we passed to it as the value of the `input_xml` parameter.
* **9:** Parsing generates events like the start or end of an element or the presence of character. There are other possible events, but these are the only ones we need to handle for our transformation. Each event provides two values, the name of the event (e.g., `START_ELEMENT`) and the value (e.g., an object of type node). We test the event type and process different types of events differently.
* **10–12:** When we start a new element, we make it a child of the last node in our stack. We seeded the stack with a new document node, called `d`, in line 6, and the first element we encounter on parsing is the root element, so we make that a child of the document node, as is normal for XML. When we push an element onto the stack, it carries its attributes with it, so unless we want to modify those (and we don’t), they’ll get taken care of automatically. When we start an element, it’s open until the corresponding `END_ELEMENT` event, so we push it onto our stack of open elements. This means that, for example, when we encounter the first child of the root element of the input XML, we’ll make that a child of the root element of the output XML that we’re constructing, since that’s the node at the top of our stack. This lets us build an XML tree for output that mirrors the input tree, except that we’ll modify it by tagging words.
* **13–14:** When we encounter an `END_ELEMENT` event, that element can’t have any more children, so we pop it off (remove it from) the top of the stack. We’ve added it to the output tree, and it remains there; popping the stack just means that it’s no longer a candidate for getting more children.
* **15–25:** We’ll illustrate how the individual lines work below, but here’s a summary with everything in one place. When we encounter `CHARACTERS` while parsing, the value of the node is an XML `text()` node, and not a string. We convert it to a plain text string with the `toxml()` method and assign it to a new variable that we call `value` (line **16**). We then use `nltk.word_tokenize()` to divide that text into a list of words (**17**). The `nltk.pos_tag()` function takes a list of words as its input and returns a list of tuples, that is, pairs of strings where the first is the original input word and the second is the part of speech according to treebank notation (**18**). We want to tag each word, so we loop over that list of tuples (**19**). For each one, we create a new `<word>` element that we assign to the variable `word` (**20**). We give the new `<word>` element an attribute with the name `@pos`, the value of which is the part of speech that `nltk.pos_tag()` returned (**21**). We also create an attribute with the name `@lemma`, the value of which is determined by using our `lemmatize()` function; we pass the text and the part of speech (after converting it from treebank to Wordnet form) into the function, and it returns the lemma (**22**). The new `<word>` element now has the two attributes we want to add, and we next create a `text()` node that uses the string value of input as its content (**23**), which we assign to the new variable `t`. We make the `text()` node a child of the new `<word>` element (**24**) and we make the `<word>` element, in turn, a child of whatever is on top of the stack (in this document, it’s a `<p>` element because that’s where all our textual input happened to be).
* **26–27:** There are other types of parse events, but we don’t need to do anything with them in this example, so we tell the parse routine to `continue` to the next parse event.

## Remind me about those NLTK functions again

### `nltk.word_tokenize()`

`nltk.word_tokenize()` splits a text into words. It’s smarter than just splitting on white space; it treats punctuation as a word, and it knows about common English contractions:

In [8]:
sample = "We didn't realize that we could split contractions!"
nltk.word_tokenize(sample)

['We',
 'did',
 "n't",
 'realize',
 'that',
 'we',
 'could',
 'split',
 'contractions',
 '!']

### `nltk.pos_tag()`

`nltk.pos_tag()` takes a list of words (not a sentence) as its input. That means that we need to tokenize the sentence before tagging:

In [9]:
sample = "We didn't realize that we could split contractions!"
words = nltk.word_tokenize(sample)
nltk.pos_tag(words)

[('We', 'PRP'),
 ('did', 'VBD'),
 ("n't", 'RB'),
 ('realize', 'VB'),
 ('that', 'IN'),
 ('we', 'PRP'),
 ('could', 'MD'),
 ('split', 'VB'),
 ('contractions', 'NNS'),
 ('!', '.')]

You can look up the part of speech tags at <https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html>.

### `nltk.stem.WordNetLemmatizer().lemmatize()`

The Wordnet lemmatizer tries to lemmatize (find the dictionary form) of a word with or without part of speech information, but without the part of speech, it guesses that everything is a noun. Remember that Wordnet knows about only nouns, verbs, adjectives, and adverbs, and that the part of speech tags are different in Wordnet than in the treebank system. Oh, and it assumes lower-case input, so if you give it a capitalized word, it won’t recognize it.

In [10]:
words = ['thing', 'things', 'Things']
[(word + ": " + nltk.stem.WordNetLemmatizer().lemmatize(word)) for word in words]

['thing: thing', 'things: thing', 'Things: Things']

In the example above, it recognizes that “thing” is the lemma for “things”, but it fails to lemmatize “Things” correctly because of the upper case letter.

In [11]:
words = [('building','n'), ('building','v')]
[(word + ": " + nltk.stem.WordNetLemmatizer().lemmatize(word, pos)) for (word, pos) in words]

['building: building', 'building: build']

In the example above, we supplied part of speech information, and the lemmatizer correctly treats “building” differently as a noun than as a verb. If we don’t specify a part of speech, it assumes everything is a noun:

In [12]:
words = ['building']
[(word + ": " + nltk.stem.WordNetLemmatizer().lemmatize(word)) for word in words]

['building: building']

____

## In case you’re curious

The following information isn’t needed for Institute activities, but if you expect to doing complex processing of XML in Python, here is a brief survey of the options.

### XML support in Python

If you’ve used XSLT (except for the new streaming facility of XSLT 3.0) or XQuery to process XML before, you’ve been doing DOM-based (Document Object Model) processing, which parses the input, builds the entire tree in memory, and operates over it. DOM-based processing makes the entire tree available at all times, which is often what you want, but it isn’t the most efficient (in terms of speed and memory) approach, so if you don’t need the entire tree at once, you may prefer an alternative. Python has support for the DOM, which you can read about in the Standard Library reference in [`xml.dom` — The Document Object Model API](https://docs.python.org/3/library/xml.dom.html) and at [Parsing XML with DOM](http://etutorials.org/Programming/Python+tutorial/Part+IV+Network+and+Web+Programming/Chapter+23.+Structured+Text+XML/23.3+Parsing+XML+with+DOM/). For constructing XML Python also provides [`xml.dom.minidom` — Minimal DOM implementation](https://docs.python.org/3/library/xml.dom.minidom.html), which “is intended to be simpler than the full DOM and also significantly smaller”. The most complete DOM resource in Python is [`xml.etree.ElementTree` — The ElementTree XML API](https://docs.python.org/3/library/xml.etree.elementtree.html#module-xml.etree.ElementTree), and the third-part [lxml](http://lxml.de/) package is further enhanced. See the note below about Installing `lxml` if you’d like to try working with it.

The primary alternative to DOM parsing in the XML world is SAX (simple API for XML; API = ‘application programming interface’), which is a streaming parser. Instead of building the entire tree in memory, a streaming parser acts on *events* like the start or end of an element, an attribute, character data, etc., and it processes each event and then moves onto the next one. In situations where you can do everything you need with an event right away, and don’t need to return to it later, SAX will be faster than DOM. Python has support for SAX processing, which you can read about in the Standard Library reference in [`xml.sax` — Support for SAX2 parsers](https://docs.python.org/3/library/xml.sax.html) or in [Parsing XML with SAX](http://etutorials.org/Programming/Python+tutorial/Part+IV+Network+and+Web+Programming/Chapter+23.+Structured+Text+XML/23.2+Parsing+XML+with+SAX/).

`pulldom`, which we use in this tutorial, has been described as follows:

> `pulldom` occupies an interesting middle ground between SAX and DOM, presenting the stream of parsing events as a Python iterator object so that you do not code callbacks, but rather loop over the events and examine each event to see if it’s of interest. When you do find an event of interest to your application, you can ask `pulldom` to build the DOM subtree rooted in that event’s node by calling method `expandNode`, and then work with that subtree as you would in `minidom`. Paul Prescod, `pulldom`’s author and XML and Python expert, describes the net result as “80% of the performance of SAX, 80% of the convenience of DOM.” ([Parsing XML with DOM](http://etutorials.org/Programming/Python+tutorial/Part+IV+Network+and+Web+Programming/Chapter+23.+Structured+Text+XML/23.3+Parsing+XML+with+DOM/))

### Installing lxml

Mac and Linux users can install `lxml` through [pypi](https://pypi.python.org/pypi/lxml/) by typing `pip install lxml`, but this installation requires developer tools that Windows users typically don’t have installed. Windows users can instead download prebuilt binaries from Christopher Gohlke’s [Unofficial Windows Binaries for Python Extension Packages](http://www.lfd.uci.edu/~gohlke/pythonlibs/#lxml) and install them by following the instructions the [Installing from wheels instructions](https://pip.pypa.io/en/latest/user_guide/#installing-from-wheels).

### About the shebang

You can run a Python program called something like `tag_words.py` from the command line by typing `python tag_words.py`. This starts the Python interpreter and tells it that the program it should interpret (and run) is a file called `tag_words.py` in the current directory. On a Unix or Mac system, though, you can run it without specifying `python` first. To do that, do the following:

1. At the command line in the directory where the program file is located, type `chmod +x tag_words.py`. The `chmod` (change mode) command sets access rights for reading, writing, and execution, and the `+x` makes this file executable. That means that it no longer needs to be executed by calling Python separately; it knows how to execute itself. You need to do this only once; the file will remain executable, even if you edit it later.
1. Since programs in many languages can be made executable, how does the system know what kind of program it is, that is, how does it know to execute this one as a Python program? That’s what the shebang does. The leading `#!` identifies the line as a shebang (only if it’s the very first line of the file), and you can read how it works at <https://stackoverflow.com/questions/43793040/how-does-usr-bin-env-work-in-a-linux-shebang-line>. 
1. Once you’ve configured the shebang and made the file executable, it’s a program you can run, but to run a program either you must tell the system where it’s located or it has to be in the execution path in your environment (you can examine that with `echo $PATH`). By default the current directory is not automatically in the path. You can either move the executable file into a directory that is in your path (a common choice is `/usr/local/bin`) or specify the path to it by prepending `./`, e.g., `./tag_words.py`. The dot means ‘current directory’, and when you supply a path to an executable, your system can run it without having to look for it on the environment execution path.

### Comments in `pulldom`

`pulldom` is supposed to be able to respond to comment events, that is, to comments (delimited by `<!--` and `-->`) in the XML input. It doesn’t work when we try it. If comments are important to your processing, you’ll need to use a parser other than `pulldom`; in this case, we happen not to care.